### Simulatore dati e streaming verso Event Hub
v1.0

In [1]:
import numpy as np
import json
import time
import datetime
import uuid

In [2]:
!pip install azure.eventhub

    100% |████████████████████████████████| 61kB 1.6MB/s ta 0:00:01
    100% |████████████████████████████████| 2.8MB 305kB/s ta 0:00:011    68% |█████████████████████▉          | 1.9MB 3.4MB/s eta 0:00:01
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from azure.eventhub import EventHubClient, Sender, EventData

Some notes:

- The batches are defined below in Batches definition (they could be generated in a loop by indicanting just the number of batches)
- All batches involve the same machine/sensors (this can be easily modified by generating the random GUID's inside the loop)
- The duration of the simulation is defined by batch_planning and batch_duration

In [4]:
# Function temperature sensor
def sensor_temp(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = {
            'batch': {
                'desc': batch['batch_description'], 
                'id': batch['ID_prefix'] + datetimeID
            },
            'id': id,
            'desc': 'temperature',
            'mu': 'C',
            'val': y,
            'time': timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
            'type': 'tbd'
            }
    return data

# Function pressure sensor
def sensor_press(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = {
            'batch': {
                'desc': batch['batch_description'], 
                'id': batch['ID_prefix'] + datetimeID
            },
            'id': id,
            'desc': 'pressure',
            'mu': 'bar',
            'val': y,
            'time': timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
            'type': 'tbd'
            }
    return data

In [5]:
# Batches definition
batches = {1: {'batch_description': 'injection piece A1', 'ID_prefix': 'INJPCA1_'},
                2: {'batch_description': 'injection piece A2', 'ID_prefix': 'INJPCA2_'},
                3: {'batch_description': 'injection piece A3', 'ID_prefix': 'INJPCA3_'},
                4: {'batch_description': 'injection piece A4', 'ID_prefix': 'INJPCA4_'},
                5: {'batch_description': 'injection piece A5', 'ID_prefix': 'INJPCA5_'}}

# Batches scheduling
batch_planning = [1, 5] #[1, 2, 3, 1, 2, 3, 1, 1] # sequence of batches = keys in batches dictionary
N_batches = len(batch_planning) # total number of batches

# Batches duration (choose batch_duration = batch_duration_fix OR batch_duration = batch_duration_random)
minutes_min, minutes_max = 1, 3 # range of batch duration in minutes (for random)
minutes_fix = 1 # batch duration in minutes (for fix)
batch_duration_fix = [1]*N_batches
batch_duration_fix = [batch_duration_fix[i]*minutes_fix for i in batch_duration_fix]
batch_duration_random = np.random.randint(minutes_min, minutes_max + 1, N_batches).tolist()
batch_duration = batch_duration_fix #batch_duration_random #batch_duration_fix

# ID producers (sensor channel identifiers, 36-digits randomly generated GUID according to standard RFC-4122)
N_channels = 4
producer_ID = [str(uuid.uuid4()) for i in range(N_channels)] # list of ID's, example 'f66aedac-9749-4491-837a-f0cb623cc2df'

# Sampling frequency (unique for all sensors)
sampling_freq = 2 # measurements/sec

In [6]:
# INFORMATION ABOUT THE CREATED EVENT HUBS NAMESPACE AND EVENT HUB
'''
ADDRESS = 'xxxxx' # example 'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1'
USER = 'RootManageSharedAccessKey'
KEY = 'xxxxx' # Primary key
client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
sender = client.add_sender(partition="0")
client.run()
'''

'\nADDRESS = \'xxxxx\' # example \'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1\'\nUSER = \'RootManageSharedAccessKey\'\nKEY = \'xxxxx\' # Primary key\nclient = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)\nsender = client.add_sender(partition="0")\nclient.run()\n'

In [8]:
# Loop generating measurements
start_time = time.time()
datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

batch_order = 0 # element of batch_planning list

while(batch_order < N_batches):

    N_batch = batch_planning[batch_order]

    # First sensor data
    data = sensor_temp(20, 30, producer_ID[0], batches[N_batch], datetimeID) # specify range for measured value
    datax = json.dumps(data)
    print(datax)
    #sender.send(EventData(datax))

    # Second sensor data
    data = sensor_press(120, 160, producer_ID[1], batches[N_batch], datetimeID) # specify range for measured value
    datax = json.dumps(data)
    print(datax)
    #sender.send(EventData(datax))

    # Add more sensors here, maximum number of sensors = N_channels
    
    time.sleep(1 / sampling_freq)

    elapsed_time = (time.time() - start_time) / 60 # minutes
    
    if elapsed_time > batch_duration[batch_order]: # in minutes
        batch_order += 1
        start_time = time.time()
        datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

#client.stop()

{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 28.17, "time": "2019-02-26 10:29:10.702", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 124.15, "time": "2019-02-26 10:29:10.702", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 24.35, "time": "2019-02-26 10:29:11.203", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 147.55, "time": "2019-02-26 10:29:11.204", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"

{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 23.76, "time": "2019-02-26 10:29:20.231", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 130.49, "time": "2019-02-26 10:29:20.231", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 21.62, "time": "2019-02-26 10:29:20.732", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 136.89, "time": "2019-02-26 10:29:20.733", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"

{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 25.38, "time": "2019-02-26 10:29:29.756", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 130.97, "time": "2019-02-26 10:29:29.757", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 22.92, "time": "2019-02-26 10:29:30.257", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 159.62, "time": "2019-02-26 10:29:30.258", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"

{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 20.22, "time": "2019-02-26 10:29:39.281", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 136.44, "time": "2019-02-26 10:29:39.281", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 21.36, "time": "2019-02-26 10:29:39.782", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 133.19, "time": "2019-02-26 10:29:39.783", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"

{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 27.18, "time": "2019-02-26 10:29:48.807", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 127.6, "time": "2019-02-26 10:29:48.808", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 28.4, "time": "2019-02-26 10:29:49.309", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 134.99, "time": "2019-02-26 10:29:49.309", "type": "tbd"}
{"batch": {"desc": "injection piece A1", "id": "INJPCA1_2019_02_26_10_29"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", 

...

{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 21.42, "time": "2019-02-26 10:30:26.918", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 123.66, "time": "2019-02-26 10:30:26.918", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 23.49, "time": "2019-02-26 10:30:27.419", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 129.21, "time": "2019-02-26 10:30:27.420", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"

{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 22.25, "time": "2019-02-26 10:30:36.444", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 144.62, "time": "2019-02-26 10:30:36.445", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 26.89, "time": "2019-02-26 10:30:36.946", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 140.3, "time": "2019-02-26 10:30:36.946", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b",

{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 21.4, "time": "2019-02-26 10:30:45.970", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 136.87, "time": "2019-02-26 10:30:45.971", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 25.47, "time": "2019-02-26 10:30:46.471", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 139.24, "time": "2019-02-26 10:30:46.472", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b",

{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 22.3, "time": "2019-02-26 10:30:55.503", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 124.91, "time": "2019-02-26 10:30:55.503", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 27.52, "time": "2019-02-26 10:30:56.007", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 138.97, "time": "2019-02-26 10:30:56.008", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b",

{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 26.29, "time": "2019-02-26 10:31:05.035", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 122.19, "time": "2019-02-26 10:31:05.036", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b", "desc": "temperature", "mu": "C", "val": 23.58, "time": "2019-02-26 10:31:05.536", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "71113d51-9e5d-4d47-adc0-0ff9401407a4", "desc": "pressure", "mu": "bar", "val": 140.66, "time": "2019-02-26 10:31:05.537", "type": "tbd"}
{"batch": {"desc": "injection piece A5", "id": "INJPCA5_2019_02_26_10_30"}, "id": "33b4b79d-ce26-452a-a7d8-c1712b88539b"